In [ ]:
import ee
import geemap
import json
from datetime import datetime

In [ ]:
# EE authentication
ee.Authenticate()
# Initialize the library.
ee.Initialize(project='ee-milagrosbecerra245')

In [ ]:
# Load the dataset
burned_area = ee.FeatureCollection("USFS/GTAC/MTBS/burned_area_boundaries/v1")
landtype = ee.ImageCollection("LANDFIRE/Vegetation/EVC/v1_4_0")
landcover = ee.ImageCollection("projects/sat-io/open-datasets/landcover/ESRI_Global-LULC_10m_TS");
burned_severity = ee.ImageCollection("USFS/GTAC/MTBS/annual_burn_severity_mosaics/v1")

# Create a Map object
Map = geemap.Map(center=[37.5, -120], zoom=6)  # Adjust center and zoom for your area of interest

# Define visualization parameters
vis_params = {
    'color': 'red'  
}

vis_params

# Add the dataset to the map
Map.addLayer(burned_area, vis_params, "Burned Area Boundaries")
Map.addLayer(landtype,)

# Display the map
Map.addLayerControl()  
Map

In [ ]:
import ee
import geemap

# Initialize the Earth Engine library
ee.Initialize()

# Load the burned area dataset
burned_area = ee.FeatureCollection("USFS/GTAC/MTBS/burned_area_boundaries/v1")

# Define the filtering function
def filter_burned_area_by_date(start_date, end_date):
    """
    Filters the burned area feature collection based on the given date range.

    Args:
        start_date (str): The start date in 'YYYY-MM-DD' format.
        end_date (str): The end date in 'YYYY-MM-DD' format.

    Returns:
        ee.FeatureCollection: Filtered feature collection.
    """
    # Convert strings to ee.Date
    start = ee.Date(start_date)
    end = ee.Date(end_date)
    
    # Filter the FeatureCollection
    filtered_burned_area = burned_area.filterDate(start, end)
    return filtered_burned_area

# Example usage
start_date = "2020-01-01"
end_date = "2021-12-31"
filtered_collection = filter_burned_area_by_date(start_date, end_date)

# Create a Map object
Map = geemap.Map(center=[37.5, -120], zoom=6)  # Adjust center and zoom for your area of interest

# Define visualization parameters
vis_params = {
    'color': 'red'  # Color for the burned area boundaries
}

# Add the filtered collection to the map
Map.addLayer(filtered_collection, vis_params, "Filtered Burned Areas")

# Add layer controls and display the map
Map.addLayerControl()
Map


In [ ]:
# Add Earth Engine dataset
dem = ee.Image("USGS/SRTMGL1_003")
landcover = ee.Image("ESA/GLOBCOVER_L4_200901_200912_V2_3").select("landcover")
landsat7 = ee.Image("LANDSAT/LE7_TOA_5YEAR/1999_2003").select(
    ["B1", "B2", "B3", "B4", "B5", "B7"]
)
states = ee.FeatureCollection("TIGER/2018/States")

# Set visualization parameters.
vis_params = {
    "min": 0,
    "max": 4000,
    "palette": ["006633", "E5FFCC", "662A00", "D8D8D8", "F5F5F5"],
}

# Add Earth Engine layers to Map
Map.addLayer(dem, vis_params, "SRTM DEM", True, 0.5)
Map.addLayer(landcover, {}, "Land cover")
Map.addLayer(
    landsat7,
    {"bands": ["B4", "B3", "B2"], "min": 20, "max": 200, "gamma": 2.0},
    "Landsat 7",
)
Map.addLayer(states, {}, "US States")

Map

In [ ]:
m = geemap.Map()

In [ ]:
url = "https://raw.githubusercontent.com/giswqs/leafmap/master/examples/data/us_cities.geojson"


In [ ]:
m.add_point_layer(url, popup=["name", "pop_max"], layer_name="US Cities")
m

In [ ]:
# Example usage of the function
start_date = '2016-01-01'
end_date = '2021-12-31'
bbox = [-124.4, 32.5, -114.1, 42.0]  # Bounding box for California
event_id = 'CA3983912034520210702'  # Replace with your desired Event ID


In [ ]:
def date_to_unix(date_str):
    """
    Convert a date string in 'YYYY-MM-DD' format to Unix timestamp in milliseconds.

    Parameters:
    - date_str (str): The date string to convert.

    Returns:
    - int: Unix timestamp in milliseconds.
    """
    return int(datetime.strptime(date_str, '%Y-%m-%d').timestamp() * 1000)

In [ ]:

start_Ig_date = date_to_unix(start_date)
end_Ig_date = date_to_unix(end_date)


# Load the MTBS burned area boundaries dataset
mtbs_boundaries = ee.FeatureCollection('USFS/GTAC/MTBS/burned_area_boundaries/v1')

# Define the AOI using the provided bbox
aoi = ee.Geometry.Rectangle(bbox)

# Filter the dataset by AOI and Ig_Date range
mtbs_boundaries_filtered = mtbs_boundaries.filterBounds(aoi).filter(
        ee.Filter.rangeContains('Ig_Date', start_Ig_date, end_Ig_date)
)

# Extract features and properties to create GeoJSON
features = mtbs_boundaries_filtered.getInfo()['features']

    # Create the GeoJSON structure
geojson = {
    "type": "FeatureCollection",
    "features": []
}

for feature in features:
    geojson['features'].append({
        "type": "Feature",
        "geometry": feature['geometry'],
        "properties": {
            "Incid_Name ": feature['properties']['Incid_Name'],
            "Ig_Date": feature['properties']['Ig_Date'],
            "Event_ID": feature['properties']['Event_ID']
        }
    })


    # Convert the GeoJSON to string format
geojson_data = json.dumps(geojson)


In [ ]:
print(geojson)

In [ ]:

# Create a map centered on the AOI
center_lat = (bbox[1] + bbox[3]) / 2
center_lon = (bbox[0] + bbox[2]) / 2
map_ = geemap.Map(center=[center_lat, center_lon], zoom=6)

# Visualization parameters
vis_params = {
    'fillColor': '#ff8a50',  # Fill color
    'color': '#ff5722',      # Outline color
    'width': 1.0             # Outline width
}

    # Add the GeoJSON layer to the map with popups
map_.addLayer(mtbs_boundaries_filtered, vis_params, popup=["Incid_Name", "Ig_Date","EventID"], layer_name="geojson_data")

    # Add the AOI boundary to the map for reference
map_.addLayer(aoi, {'color': 'blue'}, 'AOI Boundary')


In [ ]:
# Example usage of the function
start_date = '2016-01-01'
end_date = '2021-12-31'
bbox = [-124.4, 32.5, -114.1, 42.0]  # Bounding box for California
event_id = 'CA3983912034520210702'  # Replace with your desired Event ID


In [ ]:
# Call the function and display the map
display_mtbs_boundaries(bbox,start_date,end_date)

In [ ]:
import geemap
import requests

# URL to the GeoJSON for the pop-up
geojson_url = "https://apps.fs.usda.gov/arcx/rest/services/EDW/EDW_MTBS_01/MapServer/63/query?outFields=*&where=1%3D1&f=geojson"

# Load the GeoJSON
response = requests.get(geojson_url)
geojson_data = response.json()

# Filter features by year
def filter_by_year(geojson, year):
    filtered_features = [
        feature for feature in geojson['features']
        if feature['properties'].get('BurnYear') == year
    ]
    geojson['features'] = filtered_features
    return geojson

# Filter for the year 2020
filtered_geojson_2020 = filter_by_year(geojson_data, 2020)

# Display the filtered GeoJSON on the map
m = geemap.Map()  # Create a map object
m.addGeoJSON(filtered_geojson_2020, name="MTBS Pop-up GeoJSON (2020)")  # Add the filtered GeoJSON layer
m
